# P3001_Create Analytic Dataset

This notebook is for data wrangling of the Urban Ministries of Durham (UMD) homeless shelter data.

## Import Data

In [29]:
import pandas as pd                   ## for data wrangling
import numpy as np                    ## for datetime
from functools import reduce          ## for removing duplicate data

In [30]:
# import client data
client = pd.read_csv("../data/CLIENT_191102.tsv", delimiter='\t', encoding='utf-8')
entry_exit = pd.read_csv("../data/ENTRY_EXIT_191102.tsv", delimiter='\t', encoding='utf-8')
ee_udes = pd.read_csv("../data/EE_UDES_191102.tsv", delimiter='\t', encoding='utf-8')

#import data from entry
disab_entry = pd.read_csv("../data/DISABILITY_ENTRY_191102.tsv", delimiter='\t', encoding='utf-8')
health_ins_entry = pd.read_csv("../data/HEALTH_INS_ENTRY_191102.tsv", delimiter='\t', encoding='utf-8')
income_entry = pd.read_csv("../data/INCOME_ENTRY_191102.tsv", delimiter='\t', encoding='utf-8')
noncash_entry = pd.read_csv("../data/NONCASH_ENTRY_191102.tsv", delimiter='\t', encoding='utf-8')

#import data from exit
disab_exit = pd.read_csv("../data/DISABILITY_EXIT_191102.tsv", delimiter='\t', encoding='utf-8')
health_ins_exit = pd.read_csv("../data/HEALTH_INS_EXIT_191102.tsv", delimiter='\t', encoding='utf-8')
income_exit = pd.read_csv("../data/INCOME_EXIT_191102.tsv", delimiter='\t', encoding='utf-8')
noncash_exit = pd.read_csv("../data/NONCASH_EXIT_191102.tsv", delimiter='\t', encoding='utf-8')

## Data Cleaning

### General cleaning (for all datasets)

I will limit the analyses to records with EE Provider ID=Urban Ministries of Durham - Durham County - Singles Emergency Shelter - Private(5838) because the other areas have been closed.

In [31]:
subset_to_open = 'Urban Ministries of Durham - Durham County - Singles Emergency Shelter - Private(5838)'

# client data
client = client.loc[client['EE Provider ID'] == subset_to_open]
entry_exit = entry_exit.loc[entry_exit['EE Provider ID'] == subset_to_open]
ee_udes = ee_udes.loc[ee_udes['EE Provider ID'] == subset_to_open]

# data at entry
disab_entry = disab_entry.loc[disab_entry['EE Provider ID'] == subset_to_open]
health_ins_entry = health_ins_entry.loc[health_ins_entry['EE Provider ID'] == subset_to_open]
income_entry = income_entry.loc[income_entry['EE Provider ID'] == subset_to_open]
noncash_entry = noncash_entry.loc[noncash_entry['EE Provider ID'] == subset_to_open]

# data at exit
disab_exit = disab_exit.loc[disab_exit['EE Provider ID'] == subset_to_open]
health_ins_exit = health_ins_exit.loc[health_ins_exit['EE Provider ID'] == subset_to_open]
income_exit = income_exit.loc[income_exit['EE Provider ID'] == subset_to_open]

# noncash at exit has EE Provider instead of EE Provider ID
noncash_exit = noncash_exit.loc[noncash_exit['EE Provider'] == subset_to_open]

I will only going to use "Client ID" as the unique identifier in this dataset.  Therefore I will drop "Client Unique ID". "EE Provider ID" should all be the same now so I will also drop this column.

In [32]:
drop_columns=['Client Unique ID', 'EE Provider ID']
# client data
client = client.drop(drop_columns, 1)
entry_exit = entry_exit.drop(drop_columns, 1)
ee_udes = ee_udes.drop(drop_columns, 1)

# data at entry
disab_entry = disab_entry.drop(drop_columns, 1)
health_ins_entry = health_ins_entry.drop(drop_columns, 1)
income_entry = income_entry.drop(drop_columns, 1)
noncash_entry = noncash_entry.drop(drop_columns, 1)

# data at exit
disab_exit = disab_exit.drop(drop_columns, 1)
health_ins_exit = health_ins_exit.drop(drop_columns, 1)
income_exit = income_exit.drop(drop_columns, 1)

# noncash at exit has EE Provider instead of EE Provider ID
noncash_exit = noncash_exit.drop(['Client Unique ID', 'EE Provider'],1)

### Cleaning client data

In the CLIENT file, are some demographic variables of interest.  For these variables, I will convert "Doesn't know", "Refused", and "Data not collected" to missing.

In [33]:
# Remove the "(HUD)" from this response and convert don't know to missing
client['Client Primary Race']=client['Client Primary Race'].str.rstrip(" (HUD)").\
    replace("Client doesn't know", np.NaN).\
    replace("Client refused", np.NaN).\
    replace("Data not collected", np.NaN)
client.groupby("Client Primary Race").size()

client['Client Ethnicity']=client['Client Ethnicity'].str.rstrip(" (HUD)").\
    replace("Client doesn't know", np.NaN).\
    replace("Client refused", np.NaN).\
    replace("Data not collected", np.NaN)
client.groupby("Client Ethnicity").size()

client['Client Veteran Status']=client['Client Veteran Status'].str.rstrip(" (HUD)").\
    replace("Data not collected", np.NaN)
client.groupby("Client Veteran Status").size()

Client Veteran Status
No     3848
Yes     461
dtype: int64

In [34]:
## change Trans Female (MTF or Male to Female) to missing for identifiable purposes
client['Client Gender'] = client['Client Gender'].replace('Trans Female (MTF or Male to Female)', np.NaN)
client.groupby("Client Gender").size()

Client Gender
Female    1036
Male      3268
dtype: int64

In [35]:
client.head()

,EE UID,Client ID,Client Age at Entry,Client Age at Exit,Client Gender,Client Primary Race,Client Ethnicity,Client Veteran Status
0,687901,397941,60.0,61.0,Female,White,Non-Hispanic/Non-Latino,No
1,687902,130335,48.0,48.0,Female,Black or African American,Non-Hispanic/Non-Latino,No
2,687903,188933,42.0,42.0,Female,Black or African American,Non-Hispanic/Non-Latino,No
3,687904,168290,57.0,57.0,Male,White,Hispanic/Latino,No
4,687905,123122,51.0,51.0,Male,White,Non-Hispanic/Non-Latino,No


### Clean entry exit data

Of interest in this data is the length of stay at the homeless shelter.  This new variables will be calculated.

In [36]:
# select variables of interest
entry_exit = entry_exit[['EE UID', 'Entry Date', 'Exit Date', 'Destination']]

# convert columns to datetime
entry_exit[['Entry Date', 'Exit Date']] = entry_exit[['Entry Date', 'Exit Date']].apply(pd.to_datetime)

# calculate date
entry_exit['LOS']=entry_exit['Exit Date'] - entry_exit['Entry Date']

# convert length of stay to days
entry_exit["LOS"] = entry_exit["LOS"].apply(lambda row: row.days)

In [37]:
entry_exit.head()

,EE UID,Entry Date,Exit Date,Destination,LOS
0,687901,2015-08-15,2016-07-11,"Rental by client, with other ongoing housing s...",331.0
1,687902,2015-08-15,2015-08-31,Data not collected (HUD),16.0
2,687903,2015-08-15,2015-09-19,"Staying or living with friends, temporary tenu...",35.0
3,687904,2015-08-15,2016-03-07,Hospital or other residential non-psychiatric ...,205.0
4,687905,2015-08-15,2015-08-24,"Staying or living with friends, temporary tenu...",9.0


### Clean Entry Exit UDES data

This data table contains prior living situation and domestic violence.  There are over 20 different prior living situations.  Therefore I will re-categorize some of these livings situations to a smaller number of categories.  I will also remove "(HUD)" from the domestic violence response variable and change "don't know" and "refused" to missing.

In [38]:
# reclassify prior living
ee_udes['temp prior living']=ee_udes['Prior Living Situation(43)'].fillna("0")
ee_udes['Prior Living'] = pd.np.where(ee_udes['temp prior living'].str.contains("doesn't know|0|refused|not collected", case=False),"UNKNOWN",
                                      pd.np.where(ee_udes['temp prior living'].str.contains("hospital|nursing|treatment", case=False), "HOSPITAL",
                                                  pd.np.where(ee_udes['temp prior living'].str.contains("rental", case=False), "RENTAL",
                                                              pd.np.where(ee_udes['temp prior living'].str.contains("friend|family", case=False), "FRIEND or FAMILY",
                                                                          pd.np.where(ee_udes['temp prior living'].str.contains("jail", case=False), "PRISON",
                                                                                      pd.np.where(ee_udes['temp prior living'].str.contains("owned|permanent", case=False), "PERMANENT",
                                                                                                  pd.np.where(ee_udes['temp prior living'].str.contains("habitation", case=False), "NOT HABITABLE", 
                                                                                                              pd.np.where(ee_udes['temp prior living'].str.contains("transition|halfway|safe|interim|foster", case=False), "INTERIM",
                                                                                                                          pd.np.where(ee_udes['temp prior living'].str.contains("Host Home shelter"), "SHELTER","OTHER")))))))))
ee_udes.groupby("Prior Living").size()

Prior Living
FRIEND or FAMILY    1163
HOSPITAL             246
INTERIM              196
NOT HABITABLE       1313
OTHER                176
PERMANENT             38
PRISON               165
RENTAL               248
SHELTER              731
UNKNOWN               43
dtype: int64

In [39]:
# Remove the "(HUD)" from this domestic violence victim/survivor variable and combine "Client doesn't know" and "Data not collected" into "Unknown"
dv_deter_map = {"Client doesn't know (HUD)":'Unk', "Client refused (HUD)":'Unk', "No (HUD)":"No", "Yes (HUD)":"Yes"}
ee_udes['Domestic violence victim/survivor'] = ee_udes['Domestic violence victim/survivor(341)'].map(dv_deter_map)
ee_udes['Domestic violence victim/survivor'] = ee_udes['Domestic violence victim/survivor'].replace('Unk', np.NaN)
ee_udes.groupby('Domestic violence victim/survivor').size()

Domestic violence victim/survivor
No     3854
Yes     413
dtype: int64

In [40]:
# select columns of interest from ee udes
ee_udes= ee_udes[['EE UID', 'Prior Living', 'Domestic violence victim/survivor']]
ee_udes.head()

,EE UID,Prior Living,Domestic violence victim/survivor
0,687901,FRIEND or FAMILY,No
1,687902,FRIEND or FAMILY,Yes
2,687903,FRIEND or FAMILY,No
3,687904,FRIEND or FAMILY,No
4,687905,FRIEND or FAMILY,No


### Clean disability data

For the disability data, there appears to be a set number of questions asked of each client regarding a series of disabilities.  I will remove the extra '(HUD)' from the disabillity determination values and change values of 'Doesn't know' or 'Data not collected' to missing. I will also remove the extra '(HUD)' from the disabillity type.  Finally, I will transform this data from long to wide. 

In [41]:
# select variables of interest
disab_entry = disab_entry[['EE UID', 'Client ID', 'Disability Determination (Entry)',\
                           'Disability Type (Entry)', 'Date Added (417-date_added)']]

# Remove the "(HUD)" from disability determination response
# combine "Client doesn't know" and "Data not collected" into "Unknown"
disab_deter_map = {"Client doesn't know (HUD)":'Unknown', "Data not collected (HUD)":'Unknown', \
                   "No (HUD)":"No", "Yes (HUD)":"Yes"}
disab_entry['Disab Determination'] = disab_entry['Disability Determination (Entry)'].map(disab_deter_map)

# change disability determination data not collected to NaN
disab_entry['Disab Determination'] = disab_entry["Disab Determination"].replace('Unknown', np.NaN)

# Remove the "(HUD)" from disability type
disab_entry['Disability Type']=disab_entry['Disability Type (Entry)'].str.rstrip(" (HUD)")

# Drop old variables 
disab_entry=disab_entry.drop(['Disability Determination (Entry)', 'Disability Type (Entry)'], axis=1)
disab_entry.head()

,EE UID,Client ID,Date Added (417-date_added),Disab Determination,Disability Type
0,687901,397941,7/16/2015,No,Alcohol Abuse
1,687901,397941,7/16/2015,No,Both Alcohol and Drug Abuse
2,687901,397941,7/16/2015,No,Chronic Health Condition
3,687901,397941,7/16/2015,No,Developmental
4,687901,397941,7/16/2015,No,Drug Abuse


In order to transform the data, I will remove duplicate records.  That is, for records with the same entry date to the shelter, identified with a unique EE UID, I will only keep the most up to date record ie the latest Date Added, because those records look like they were updated.

In [42]:
# sorting by unique shelter visit id, client id, disability type, and date added (if any updates)
disab_entry.sort_values(by=['EE UID', 'Client ID', 'Disability Type', 'Date Added (417-date_added)'], inplace=True)

# dropping duplicate values - we will only keep the last dated record because this looks to me like it was an "update"
disab_entry.drop_duplicates(subset=['EE UID', 'Client ID', 'Disability Type'], keep='first',inplace=True)

# drop date variable because it's no longer needed
disab_entry=disab_entry.drop(['Date Added (417-date_added)'], axis=1)

#Transform data so 1 column for each disability type and disab determination as the values.
disab_entry_t = disab_entry.pivot(index='EE UID', columns='Disability Type', values='Disab Determination')
disab_entry_t.head()

Disability Type,Alcohol Abuse,Both Alcohol and Drug Abuse,Chronic Health Condition,Developmental,Drug Abuse,Dual Diagnosis,HIV/AIDS,Hearing Impaired,Mental Health Problem,Other,Other: Learning,Other: Speech,Physical,Physical/Medical,Vision Impaired
EE UID,,,,,,,,,,,,,,,
687901,No,No,No,No,No,NaN,No,NaN,Yes,NaN,NaN,NaN,No,Yes,NaN
687902,No,No,Yes,No,No,NaN,No,NaN,No,NaN,NaN,NaN,No,Yes,NaN
687903,No,No,No,No,Yes,NaN,No,NaN,Yes,NaN,NaN,NaN,Yes,NaN,NaN
687904,No,No,Yes,No,No,NaN,No,NaN,No,NaN,NaN,NaN,No,NaN,NaN
687905,No,No,Yes,Yes,No,NaN,No,NaN,Yes,NaN,NaN,NaN,No,Yes,NaN


Finally, I will create a variable that indicates if there were ANY disability.

In [43]:
disab_entry_t['Any Disability']="No"
for index in disab_entry_t.index:
    any_disability="No"
    for col in disab_entry_t.columns:
        if disab_entry_t[col][index] == "Yes":
            any_disability="Yes"
    disab_entry_t['Any Disability'][index]=any_disability
disab_entry_t.head()

Disability Type,Alcohol Abuse,Both Alcohol and Drug Abuse,Chronic Health Condition,Developmental,Drug Abuse,Dual Diagnosis,HIV/AIDS,Hearing Impaired,Mental Health Problem,Other,Other: Learning,Other: Speech,Physical,Physical/Medical,Vision Impaired,Any Disability
EE UID,,,,,,,,,,,,,,,,
687901,No,No,No,No,No,NaN,No,NaN,Yes,NaN,NaN,NaN,No,Yes,NaN,Yes
687902,No,No,Yes,No,No,NaN,No,NaN,No,NaN,NaN,NaN,No,Yes,NaN,Yes
687903,No,No,No,No,Yes,NaN,No,NaN,Yes,NaN,NaN,NaN,Yes,NaN,NaN,Yes
687904,No,No,Yes,No,No,NaN,No,NaN,No,NaN,NaN,NaN,No,NaN,NaN,Yes
687905,No,No,Yes,Yes,No,NaN,No,NaN,Yes,NaN,NaN,NaN,No,Yes,NaN,Yes


### Clean health insurance data

For the health insurance data, there appears to be a set number of questions asked of each client regarding a series of health insurances, like the disability data.  I will first change values of 'Doesn't know' or 'Data not collected' to missing. I will transform this data from long to wide. 

In [44]:
# change data not collected to NaN
health_ins_entry['Covered'] = health_ins_entry["Covered (Entry)"].replace('Data Not Collected', np.NaN)

# sorting by unique shelter visit id, client id, disability type, and date added (if any updates)
health_ins_entry.sort_values(by=['EE UID', 'Client ID', 'Health Insurance Type (Entry)', \
                                 'Date Added (4307-date_added)'], inplace=True)

# dropping duplicate values - we will only keep the last dated record because this looks to me like it was an "update"
health_ins_entry.drop_duplicates(subset=['EE UID', 'Client ID', 'Health Insurance Type (Entry)'], keep='first',inplace=True)

# select variables of interest
health_ins_entry=health_ins_entry[['EE UID', 'Covered', 'Health Insurance Type (Entry)']]

# delete entries where health insurance type is NAN - all of these have covered values = nan too
health_ins_entry=health_ins_entry.dropna(subset=['Health Insurance Type (Entry)'])

#Transform data so 1 column for each insurance type and covered entry as the values.
health_ins_entry_t = health_ins_entry.pivot(index='EE UID', columns='Health Insurance Type (Entry)', values='Covered')
health_ins_entry_t.head()

Health Insurance Type (Entry),Employer - Provided Health Insurance,Health Insurance obtained through COBRA,Indian Health Services Program,MEDICAID,MEDICARE,Other,Private Pay Health Insurance,State Children's Health Insurance Program,State Health Insurance for Adults,Veteran's Administration (VA) Medical Services
EE UID,,,,,,,,,,
687901,No,No,No,No,No,No,No,No,No,No
687902,No,No,NaN,Yes,No,NaN,No,No,No,No
687903,No,No,NaN,No,No,NaN,No,No,No,No
687904,No,No,NaN,No,No,NaN,No,No,No,No
687905,No,No,NaN,No,No,NaN,No,No,No,No


Finally, I will create a variable that indicates if there were ANY health insurance.

In [45]:
health_ins_entry_t['Any Health Insurance']="No"
for index in health_ins_entry_t.index:
    any_ins="No"
    for col in health_ins_entry_t.columns:
        if health_ins_entry_t[col][index] == "Yes":
            any_ins="Yes"
    health_ins_entry_t['Any Health Insurance'][index]=any_ins
health_ins_entry_t.head()

Health Insurance Type (Entry),Employer - Provided Health Insurance,Health Insurance obtained through COBRA,Indian Health Services Program,MEDICAID,MEDICARE,Other,Private Pay Health Insurance,State Children's Health Insurance Program,State Health Insurance for Adults,Veteran's Administration (VA) Medical Services,Any Health Insurance
EE UID,,,,,,,,,,,
687901,No,No,No,No,No,No,No,No,No,No,No
687902,No,No,NaN,Yes,No,NaN,No,No,No,No,Yes
687903,No,No,NaN,No,No,NaN,No,No,No,No,No
687904,No,No,NaN,No,No,NaN,No,No,No,No,No
687905,No,No,NaN,No,No,NaN,No,No,No,No,No


### Clean income data

For the income data, there appears to be a set number of questions asked of each client regarding any income.  I will first change values of 'Doesn't know' or 'Data not collected' to missing. Then I will transform this data from long to wide. 

In [46]:
# Remove the "(HUD)" from this response
income_entry['Income Source']=income_entry['Income Source (Entry)'].str.rstrip(" (HUD)")

# change data not collected to NaN
income_entry['Receiving Income'] = income_entry["Receiving Income (Entry)"].replace('Data Not Collected', np.NaN)

# sorting by unique shelter visit id, client id, disability type, and date added (if any updates)
income_entry.sort_values(by=['EE UID', 'Client ID', 'Income Source', 'Date Added (140-date_added)'], inplace=True)

# dropping duplicate values - we will only keep the last dated record because this looks to me like it was an "update"
income_entry.drop_duplicates(subset=['EE UID', 'Client ID', 'Income Source'], keep='first',inplace=True)

# keep variables of interest
income_entry=income_entry[['EE UID', 'Receiving Income', 'Income Source']]

# delete entries where income type is NAN - all of these have covered values = nan too
income_entry=income_entry.dropna(subset=['Income Source'])

#Transform data so 1 column for each insurance type and covered entry as the values.
income_entry_t = income_entry.pivot(index='EE UID', columns='Income Source', values='Receiving Income')
income_entry_t.head()

Income Source,Alimony or Other Spousal Support,Child Support,Earned Income,General Assistance,No Financial Resources,Other,Pension or retirement income from another job,Private Disability Insurance,Retirement Income From Social Security,SSDI,SSI,TANF,Unemployment Insurance,VA Non-Service Connected Disability Pension,VA Service Connected Disability Compensation,Worker's Compensation
EE UID,,,,,,,,,,,,,,,,
687901,No,No,No,No,NaN,No,No,No,No,No,No,No,No,No,No,No
687902,No,No,No,No,NaN,No,No,No,No,No,Yes,No,No,No,No,No
687903,No,No,No,No,NaN,No,No,No,No,No,No,No,No,No,No,No
687904,No,No,No,No,NaN,No,No,No,No,No,No,No,No,No,No,No
687905,No,No,No,No,NaN,No,No,No,No,No,No,No,No,No,No,No


Finally, I will create a variable that indicates if there were ANY income.

In [47]:
income_entry_t['Any Income Source']="No"
for index in income_entry_t.index:
    any_income="No"
    for col in income_entry_t.columns:
        if income_entry_t[col][index] == "Yes":
            any_income="Yes"
    income_entry_t['Any Income Source'][index]=any_income
income_entry_t.head()

Income Source,Alimony or Other Spousal Support,Child Support,Earned Income,General Assistance,No Financial Resources,Other,Pension or retirement income from another job,Private Disability Insurance,Retirement Income From Social Security,SSDI,SSI,TANF,Unemployment Insurance,VA Non-Service Connected Disability Pension,VA Service Connected Disability Compensation,Worker's Compensation,Any Income Source
EE UID,,,,,,,,,,,,,,,,,
687901,No,No,No,No,NaN,No,No,No,No,No,No,No,No,No,No,No,No
687902,No,No,No,No,NaN,No,No,No,No,No,Yes,No,No,No,No,No,Yes
687903,No,No,No,No,NaN,No,No,No,No,No,No,No,No,No,No,No,No
687904,No,No,No,No,NaN,No,No,No,No,No,No,No,No,No,No,No,No
687905,No,No,No,No,NaN,No,No,No,No,No,No,No,No,No,No,No,No


### Clean Noncash data

For the noncash income data, there appears to be a set number of questions asked of each client regarding any noncash income.  I will first change values of 'Doesn't know' or 'Data not collected' to missing. Then I will transform this data from long to wide. 

In [48]:
# Remove the "(HUD)" from this response
noncash_entry['Noncash Source']=noncash_entry['Non-Cash Source (Entry)'].str.rstrip(" (HUD)")

# change data not collected to NaN
noncash_entry['Receiving Benefit'] = noncash_entry["Receiving Benefit (Entry)"].replace('Data Not Collected', np.NaN)

# sorting by unique shelter visit id, client id, disability type, and date added (if any updates)
noncash_entry.sort_values(by=['EE UID', 'Client ID', 'Noncash Source', 'Date Added (2704-date_added)'], inplace=True)

# dropping duplicate values - we will only keep the last dated record because this looks to me like it was an "update"
noncash_entry.drop_duplicates(subset=['EE UID', 'Client ID', 'Noncash Source'], keep='first',inplace=True)

# keep variables of interest
noncash_entry=noncash_entry[['EE UID', 'Receiving Benefit', 'Noncash Source']]

# delete entries where income type is NAN - all of these have covered values = nan too
noncash_entry=noncash_entry.dropna(subset=['Noncash Source'])

#Transform data so 1 column for each insurance type and covered entry as the values.
noncash_entry_t = noncash_entry.pivot(index='EE UID', columns='Noncash Source', values='Receiving Benefit')
noncash_entry_t.head()

Noncash Source,Other Source,Other TANF-Funded Services,"Section 8, Public Housing, or other ongoing rental assistance",Special Supplemental Nutrition Program for WIC,Supplemental Nutrition Assistance Program (Food Stamps,TANF Child Care Services,TANF Transportation Services,Temporary rental assistance
EE UID,,,,,,,,
687901,No,No,No,No,Yes,No,No,No
687902,No,No,No,No,No,No,No,No
687903,No,No,No,No,NaN,No,No,No
687904,No,No,No,No,No,No,No,No
687905,No,No,No,No,Yes,No,No,No


Finally, I will create a variable that indicates if there were ANY noncash income.

In [49]:
noncash_entry_t['Any Noncash Source']="No"
for index in noncash_entry_t.index:
    any_noncash="No"
    for col in noncash_entry_t.columns:
        if noncash_entry_t[col][index] == "Yes":
            any_noncash="Yes"
    noncash_entry_t['Any Noncash Source'][index]=any_noncash
noncash_entry_t.head()

Noncash Source,Other Source,Other TANF-Funded Services,"Section 8, Public Housing, or other ongoing rental assistance",Special Supplemental Nutrition Program for WIC,Supplemental Nutrition Assistance Program (Food Stamps,TANF Child Care Services,TANF Transportation Services,Temporary rental assistance,Any Noncash Source
EE UID,,,,,,,,,
687901,No,No,No,No,Yes,No,No,No,Yes
687902,No,No,No,No,No,No,No,No,No
687903,No,No,No,No,NaN,No,No,No,No
687904,No,No,No,No,No,No,No,No,No
687905,No,No,No,No,Yes,No,No,No,Yes


## Merge all data to create analytic dataset

In [50]:
# merge all data frames together to create an analytic dataset with 1 record per EE UID
data_frames = [client, entry_exit, ee_udes, disab_entry_t, health_ins_entry_t, income_entry_t, noncash_entry_t]
anl = reduce(lambda  left,right: pd.merge(left,right,on=['EE UID'], how='left'), data_frames)

# sort values by client id and entry date
anl.sort_values(by=['Client ID', 'Entry Date'], inplace=True)

# output first record only to get the FIRST entry of shelter use for a client
anl_first = anl.drop_duplicates(subset='Client ID', keep='first')

# output data to tsv files for data analysis in R
anl.to_csv("../data/analytic.tsv", sep='\t')
anl_first.to_csv("../data/analytic_first.tsv", sep='\t')